<a href="https://colab.research.google.com/github/joshuajhchoi/ai2learn/blob/master/Faster_RCNN_pytorch_notebook_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Faster-RCNN-with-torchvision-master

/content/drive/MyDrive/Faster-RCNN-with-torchvision-master


In [ ]:
import utils
import dataset.transforms as T
import datetime
import os
import time

import torch
import torch.utils.data
from torch import nn
from dataset.coco_utils import get_coco, get_coco_kp
from engine import train_one_epoch, evaluate
from dataset.group_by_aspect_ratio import GroupedBatchSampler, create_aspect_ratio_groups
import argparse
import torchvision

import cv2
import random


In [ ]:
data_path = '../../data/coco/'
model_name = 'fasterrcnn_resnet50_fpn'
dataset_name = 'coco'
device_type = 'cuda'
batch_size = 8
epochs = 20
workers = 8
lr = 0.02
momentum = 0.9
weight_decay = 0.0001
print_freq = 20
lr_step_size = 8
lr_steps = [8,11]
lr_gamma = 0.1
resume = ''
test_only = False
output_dir = 'result'
aspect_ratio_group_factor = 0
pretrained = True
distributed = False
parallel = False
world_size =1
dist_url = 'env://'



In [ ]:
def get_dataset(name, image_set, transform):
    paths = {
        "coco": ('coco/', get_coco, 91)
        # "coco_kp": ('/datasets01/COCO/022719/', get_coco_kp, 2)
    }
    p, ds_fn, num_classes = paths[name]

    ds = ds_fn(p, image_set=image_set, transforms=transform)
    return ds, num_classes

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:

        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
def main():
    if output_dir:
        utils.mkdir(output_dir)

    # Data loading
    print("Loading data")
    dataset, num_classes = get_dataset(dataset_name, "val", get_transform(train=True))# to train on training set change "val" to "train"
    dataset_test, _ = get_dataset(dataset_name, "val", get_transform(train=False))

    print("Creating data loaders")
    if distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(dataset)
        test_sampler = torch.utils.data.distributed.DistributedSampler(dataset_test)
    else:
        train_sampler = torch.utils.data.RandomSampler(dataset)
        test_sampler = torch.utils.data.SequentialSampler(dataset_test)

    if aspect_ratio_group_factor >= 0:
        group_ids = create_aspect_ratio_groups(dataset, k=aspect_ratio_group_factor)
        train_batch_sampler = GroupedBatchSampler(train_sampler, group_ids, batch_size)
    else:
        train_batch_sampler = torch.utils.data.BatchSampler(
            train_sampler, batch_size, drop_last=True)

    data_loader = torch.utils.data.DataLoader(
        dataset, batch_sampler=train_batch_sampler, num_workers=workers,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=batch_size,
        sampler=test_sampler, num_workers=workers,
        collate_fn=utils.collate_fn)

    # Model creating
    print("Creating model")
    # model = models.__dict__[model](num_classes=num_classes, pretrained=pretrained)
    model = torchvision.models.detection.__dict__[model_name](num_classes=num_classes,
                                                              pretrained=pretrained)

    device = torch.device(device_type)
    model.to(device)

    # Distribute
    model_without_ddp = model
    if distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[gpu])
        model_without_ddp = model.module

    # Parallel
    if parallel:
        print('Training parallel')
        model = torch.nn.DataParallel(model).cuda()
        model_without_ddp = model.module

    # Optimizer
    params = [p for p in model.parameters() if p.requires_grad]

    optimizer = torch.optim.SGD(
        params, lr=lr, momentum=momentum, weight_decay=weight_decay)

    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=lr_steps, gamma=lr_gamma)

    # Resume training
    if resume:
        print('Resume training')
        checkpoint = torch.load(resume, map_location='cpu')
        model_without_ddp.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])

    if test_only:
        evaluate(model, data_loader_test, device=device)
        return

    # Training
    print('Start training')
    start_time = time.time()
    for epoch in range(epochs):
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq)
        lr_scheduler.step()
        if output_dir:
            utils.save_on_master({
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict()
                },
                os.path.join(output_dir, 'model_{}.pth'.format(epoch)))

        # evaluate after every epoch
        evaluate(model, data_loader_test, device=device)

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))

#Download Sample Coco Dataset

In [ ]:
# To train on full dataset uncomment, commented lines
# By default training will use val dataset due to size constraints
!mkdir -p coco
# !wget http://images.cocodataset.org/zips/train2017.zip -P coco/
!wget http://images.cocodataset.org/zips/val2017.zip -P coco/
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P coco/
!unzip coco/annotations_trainval2017.zip -d coco/
!unzip coco/val2017.zip -d coco/
# !unzip coco/train2017.zip -d coco/


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: coco/val2017/000000212226.jpg  
 extracting: coco/val2017/000000231527.jpg  
 extracting: coco/val2017/000000578922.jpg  
 extracting: coco/val2017/000000062808.jpg  
 extracting: coco/val2017/000000119038.jpg  
 extracting: coco/val2017/000000114871.jpg  
 extracting: coco/val2017/000000463918.jpg  
 extracting: coco/val2017/000000365745.jpg  
 extracting: coco/val2017/000000320425.jpg  
 extracting: coco/val2017/000000481404.jpg  
 extracting: coco/val2017/000000314294.jpg  
 extracting: coco/val2017/000000335328.jpg  
 extracting: coco/val2017/000000513688.jpg  
 extracting: coco/val2017/000000158548.jpg  
 extracting: coco/val2017/000000132116.jpg  
 extracting: coco/val2017/000000415238.jpg  
 extracting: coco/val2017/000000321333.jpg  
 extracting: coco/val2017/000000081738.jpg  
 extracting: coco/val2017/000000577584.jpg  
 extracting: coco/val2017/000000346905.jpg  
 extracting: coco/val2017/000000433980.jpg  
 extracting: coco/v

In [ ]:
main()

Loading data
loading annotations into memory...
Done (t=0.81s)
creating index...
index created!
loading annotations into memory...
Done (t=0.76s)
creating index...
index created!
Creating data loaders
Using [0, 1.0, inf] as bins for aspect ratio quantization
Count of instances per bin: [3602 1398]
Creating model


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. Yo

Start training
Epoch: [0]  [  0/625]  eta: 1:21:44  lr: 0.000052  loss: 0.5723 (0.5723)  loss_classifier: 0.2473 (0.2473)  loss_box_reg: 0.2300 (0.2300)  loss_objectness: 0.0327 (0.0327)  loss_rpn_box_reg: 0.0623 (0.0623)  time: 7.8466  data: 2.3779  max mem: 10392
Epoch: [0]  [ 20/625]  eta: 0:24:06  lr: 0.000692  loss: 0.5693 (0.6058)  loss_classifier: 0.2386 (0.2383)  loss_box_reg: 0.2450 (0.2541)  loss_objectness: 0.0351 (0.0611)  loss_rpn_box_reg: 0.0483 (0.0524)  time: 2.1174  data: 0.0485  max mem: 11821
Epoch: [0]  [ 40/625]  eta: 0:22:11  lr: 0.001333  loss: 0.5351 (0.6100)  loss_classifier: 0.2134 (0.2323)  loss_box_reg: 0.2430 (0.2507)  loss_objectness: 0.0348 (0.0715)  loss_rpn_box_reg: 0.0414 (0.0555)  time: 2.1558  data: 0.0454  max mem: 11821
Epoch: [0]  [ 60/625]  eta: 0:21:25  lr: 0.001973  loss: 0.5572 (0.6027)  loss_classifier: 0.2264 (0.2305)  loss_box_reg: 0.2255 (0.2456)  loss_objectness: 0.0427 (0.0672)  loss_rpn_box_reg: 0.0704 (0.0595)  time: 2.2738  data: 0.04